# Anomaly Detection using VAR

In [2]:
# from google.colab import files
# from google.colab import drive
import os
import io
import pandas as pd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chisquare
import numpy as np

from datetime import datetime

%matplotlib inline
import seaborn as sns
sns.set_style("darkgrid")
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score

import optuna
from optuna import Trial
from optuna.samplers import TPESampler

from statsmodels.tsa.stattools import adfuller

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_Gangwon = pd.read_csv('../data/train_Gangwon.csv', index_col = 0)
test_Gangwon = pd.read_csv('../data/test_Gangwon.csv', index_col = 0)
train_Gangwon_y = pd.read_csv('../data/train_Gangwon_y.csv', index_col = 0)
test_Gangwon_y = pd.read_csv('../data/test_Gangwon_y.csv', index_col = 0)

train_Seoul = pd.read_csv('../data/train_Seoul.csv', index_col = 0)
test_Seoul = pd.read_csv('../data/test_Seoul.csv', index_col = 0)
train_Seoul_y = pd.read_csv('../data/train_Seoul_y.csv', index_col = 0)
test_Seoul_y = pd.read_csv('../data/test_Seoul_y.csv', index_col = 0)

train_Busan = pd.read_csv('../data/train_Busan.csv', index_col = 0)
test_Busan = pd.read_csv('../data/test_Busan.csv', index_col = 0)
train_Busan_y = pd.read_csv('../data/train_Busan_y.csv', index_col = 0)
test_Busan_y = pd.read_csv('../data/test_Busan_y.csv', index_col = 0)

In [10]:
train_Gangwon

,avg_ta,ca,sum_rn,comp_wd,comp_ws
0,-1.400564,0.330668,-0.320777,-2.104921,-1.048927
17,-1.689726,-0.696407,-0.329270,-1.214750,1.378073
34,-1.622255,-0.035748,-0.330533,-1.914140,-0.810434
51,-1.805391,-1.154427,-0.335123,-1.837161,-1.248643
68,-1.969250,-1.741526,-0.334536,-1.244955,0.129552
...,...,...,...,...,...
62016,-1.959611,-1.137376,-0.333976,-1.273244,-1.693817
62033,-1.487313,-0.530350,-0.335123,-1.136252,0.617652
62050,-1.111402,0.338301,-0.318137,-1.472649,-2.664797
62067,-1.439119,-1.464731,-0.335123,-2.794641,-4.198421


In [4]:
def test_stationarity(ts_data, column='', signif=0.05, series=False):
    if series:
        adf_test = adfuller(ts_data, autolag='AIC')
    else:
        adf_test = adfuller(ts_data[column], autolag='AIC')
    p_value = adf_test[1]
    if p_value <= signif:
        test_result = "Stationary"
    else:
        test_result = "Non-Stationary"
    return test_result

신뢰성 제로다 진짜....

In [16]:
adf_test_results = {
    col: test_stationarity(train_Gangwon, col)
    for col in train_Gangwon.columns
}
adf_test_results

{'avg_ta': 'Stationary',
 'ca': 'Stationary',
 'sum_rn': 'Stationary',
 'comp_wd': 'Stationary',
 'comp_ws': 'Stationary'}

In [5]:
from statsmodels.tsa.api import VAR

In [18]:
max_lag = 365
var_model = VAR(train_Gangwon)
# select the best lag order
lag_results = var_model.select_order(max_lag)
selected_lag = lag_results.aic
print(selected_lag) # selected_lag = 13

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


7


In [7]:
selected_lag = 7
def find_anomalies(squared_errors):
    threshold = np.mean(squared_errors) + np.std(squared_errors)
    predictions = (squared_errors >= threshold).astype(int)
    return predictions, threshold
var = VAR(test_Gangwon)
var_fitresults = var.fit(selected_lag)
squared_errors = var_fitresults.resid.sum(axis=1) ** 2
predictions, threshold = find_anomalies(squared_errors) # threshold = 7593.829254818655

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [9]:
threshold

18.708409658490755

In [19]:
prediction = pd.concat([pd.DataFrame([0,0,0,0,0,0,0]),predictions], axis = 0)

In [20]:
from sklearn.metrics import f1_score

f1 = f1_score(test_Gangwon_y, prediction,average='micro')
print('f1 score :', f1)

f1 score : 0.875


In [15]:
type(predictions)

pandas.core.series.Series

In [1]:
17*7

119